In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np
import json
import pyodbc
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL

# Connect to Database

In [2]:
user_pass = pd.read_csv(r"C:\Users\stgadmin\Desktop\Data_digicity\05_Librairies_python\04_SQL_librairies\SQL_pass.csv")
#user_pass

In [3]:
# Connection server
server = user_pass.at[0,"server"]
username = user_pass.at[0,"user"]
password = user_pass.at[0,"mdp"]

connect_string = ('DRIVER={SQL Server};SERVER='
                     +server
                     +';Trusted_Connection=yes;UID='
                     +username
                     +';PWD='
                     + password
                    )

In [4]:
# Connection server + DB
server = user_pass.at[0,"server"]
username = user_pass.at[0,"user"]
password = user_pass.at[0,"mdp"]
database = "BelgiumPopulation"

connect_string = 'DRIVER={SQL Server};SERVER='+server+";DATABASE="+database+';Trusted_Connection=yes;UID='+username+';PWD='+ password

In [5]:
# Use pyodbc with sqlalchemy
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connect_string})
engine = create_engine(connection_url) #+-= cnxn= pyodbc.connect(connection_string)

# Load tables

In [6]:
FactsBEPop = """
                SELECT *
                FROM FactsBEPop
            """

In [9]:
PopByNationality = pd.read_sql_query(FactsBEPop,con=engine)

# Unpivot large tables and export to CSV

In [74]:
PopByNat_melted = PopByNationality.melt(id_vars=["Id_data", "CODE INS", "Lieu de résidence", "Province", "Région", "Année", "Sexe"],
                                        var_name = "Pays d'origine",
                                        value_name= "Citoyens répertoriés")

In [76]:
PopByNat_melted = PopByNat_melted[PopByNat_melted["Lieu de résidence"] != "Province du Luxembourg"]

In [87]:
PopByNat_melted

,Id_data,CODE INS,Lieu de résidence,Province,Région,Année,Sexe,Pays d'origine,Citoyens répertoriés
0,1,11001,Aartselaar,Anvers,Flandre,1992,HOMME,Belgique,6935
1,2,11001,Aartselaar,Anvers,Flandre,1992,FEMME,Belgique,7096
2,3,11002,Anvers,Anvers,Flandre,1992,HOMME,Belgique,193669
3,4,11002,Anvers,Anvers,Flandre,1992,FEMME,Belgique,216987
4,5,11004,Boechout,Anvers,Flandre,1992,HOMME,Belgique,5543
...,...,...,...,...,...,...,...,...,...
4150097,38782,93056,Philippeville,Namur,Wallonie,2024,FEMME,Indéterminés/Apatrides,2
4150098,38783,93088,Walcourt,Namur,Wallonie,2024,HOMME,Indéterminés/Apatrides,1
4150099,38784,93088,Walcourt,Namur,Wallonie,2024,FEMME,Indéterminés/Apatrides,0
4150100,38785,93090,Viroinval,Namur,Wallonie,2024,HOMME,Indéterminés/Apatrides,1


In [78]:
PopByNat_melted.to_csv("./Population_par_origine_1992-2024.csv", 
                       index=False, 
                       columns=["CODE INS", "Lieu de résidence", "Province", "Région", "Année", "Sexe", "Pays d'origine", "Citoyens répertoriés"])

# Missing info

In [48]:
PopByNationality["Sexe"].value_counts()

Sexe
HOMME    19393
FEMME    19393
Name: count, dtype: int64

In [49]:
femmes = PopByNationality[PopByNationality["Sexe"] == "FEMME"]

In [50]:
# Missing data as from 2019 (581 entries) and then as from 2021 (582 entries)
# Reason: fusion of some communes into new communes 
count_data = femmes.groupby(by="Année", as_index=False)[["CODE INS", "Lieu de résidence"]].count()
count_data

,Année,CODE INS,Lieu de résidence
0,1992,589,589
1,1993,589,589
2,1994,589,589
3,1995,589,589
4,1996,589,589
5,1997,589,589
6,1998,589,589
7,1999,589,589
8,2000,589,589
9,2001,589,589


In [80]:
Pop_1992 = PopByNationality[PopByNationality["Année"] == 1992]
Pop_1992 = Pop_1992[Pop_1992["Lieu de résidence"] != "Province du Luxembourg"]
Pop_2019 = PopByNationality[PopByNationality["Année"] == 2019]
Pop_2019 = Pop_2019[Pop_2019["Lieu de résidence"] != "Province du Luxembourg"]

In [81]:
residence_1992 = Pop_1992["Lieu de résidence"].unique()
residence_2019 = Pop_2019["Lieu de résidence"].unique()

In [83]:
print(f"""Nombre de lieux uniques en 1992 : {len(residence_1992)} \n
Nombre de lieux uniques en 2019 : {len(residence_2019)}""")

Nombre de lieux uniques en 1992 : 589 

Nombre de lieux uniques en 2019 : 581


In [84]:
set(residence_1992) - set(residence_2019)

{'Blégny',
 'Knesselare',
 'Kruishoutem',
 'Lovendegem',
 'Meeuwen-Gruitrode',
 'Neerpelt',
 'Nevele',
 'Opglabbeek',
 'Overpelt',
 'Puurs',
 'Sint-Amands',
 'Waarschoot',
 'Zingem',
 'Zomergem',
 'Zwalm'}

In [85]:
set(residence_2019) - set(residence_1992)

{'Blegny',
 'Kruisem',
 'Lievegem',
 'Oudsbergen',
 'Pelt',
 'Puurs-Sint-Amands',
 'Zwalin'}